# MS-E2121 - Linear Optimization

## Homework 3

In [25]:
using LinearAlgebra

## Revised simplex
The following cell contains all the functions that are used in our implementation of the revised simplex method. Your task is to implement the functions by replacing the "TODO: add your code here" -comments with your code.

In [26]:
## Dictionary of elements:
#     A: The constraint matrix
#     b: The RHS (right-hand side) vector
#     B_hist: Column indices of the basic variables at each iteration
#     B_ind: The column indices of the basic variables
#     c: The vector of cost coefficients
#     cᵣ: The vector of reduced costs
#     c_hist: Reduced costs at each iteration
#     I: contains the row indices of u for which its components are positive    
#     iter:   Number of iterations so far
#     J: set of indices of nonbasic variables that are candidate to become basic (with cᵣ < 0)
#     j_in: index of the nonbasic variable selected to become basic
#     l_out: the index of the basic variable selected to become nonbasic  
#     rule: the pivoting rule for choosing the entering variable. Options are :bland and :dantzig 
#     u: the reverse of the basic directions 
#     x_hist: Decision variable values at each iteration

## Update "history arrays"
function update_history!(c_hist, cᵣ, x_hist, x, B_hist, B_ind, iter)
    c_hist[iter,:] = cᵣ
    x_hist[iter,:] = x
    B_hist[iter,:] = B_ind
end

## Return true if the while-loop should continue, 
## false if optimal solution has been obtained
function iteration_condition(cᵣ, ϵ)
    return !all(cᵢ>=-ϵ for cᵢ in cᵣ)
end

## Calculate the reduced costs cᵣ (Hint: you might want to calculate p, the vector of dual variables, 
## but it doesn't need to be returned as it is not used in other parts of the algorithm)
function calculate_cr(A, c, B_ind, invB)
    # TODO: add your code here
    p = c[B_ind]'*invB
    cᵣ = c' - p*A
    return cᵣ
end

## Choose the variable with the smallest column index in accordance with predefined rule
function select_entering_variable(J, cᵣ, rule)
    if rule == :dantzig   # Dantzig's rule: most negative reduced cost
        # TODO: add your code here
        j_in = argmin(cᵣ)[2]
    elseif rule == :bland # Bland's rule: smallest index j    
        # TODO: add your code here
        j_in = minimum(J)
    end
    return j_in
end

## Compute the vector u (i.e., the reverse of the basic directions)
function calculate_reverse_basic_directions(A, invB, j_in)
    # TODO: add your code here
    u = invB*A[:,j_in]
    return u
end

## Compute the optimal step length θ by performing element-wise
## divisions (./) between the vector x(B_ind) of basic variables and the vector
## u. The divisions are performed only for the elements of u and x(B_ind(I))
## with indices in I (I contains the row indices of u for which
## u(I) is positive). The corresponding basic variables are in vector x(B_ind(I)).
## The minimum of these element-wise divisions is the optimal step length.
function calculate_step_length(x, B_ind, u, I) 
    # TODO: add your code here
    θ = minimum(x[B_ind[I]] ./ u[I])
    return θ
end

## Select the smallest (row) index among the potential exiting variables in
## accordance with Bland's rule (minimum index) and store it to l_out
## Hint: First find the row indices of all the basic variables with the optimal step length θ
## by performing element-wise divisions (./) between the vector of basic
## variables x(B_ind) and the vector u and checking if the step equals θ.
## This set (of those indices for which the step is equal to θ) then contains all the basic row indices 
## with the optimal step length (i.e., the row indices of the basic variables that can exit the basis)
function select_leaving_variable(B_ind, x, u, I, θ)
    # TODO: add your code here
    step_lengths = x[B_ind] ./ u
    l_out = minimum(findall(step_lengths .== θ))
    return l_out #B_ind[l_out]
end

## Move to the adjacent solution (move in direction -u with the optimal step 
## length θ): x(B) = x(B) - θ*u    AND   x(j) = 0 + θ
## NOTE: The exclamation mark is a naming convention for a mutating function,
## that is, a function that modifies its input variables and doesn't necessarily return anything
## See for example the function update_history! above
function update_x!(x, j_in, B_ind, θ, u)
    # TODO: add your code here
    x[B_ind] = x[B_ind] - θ*u
    x[j_in] = θ
end

## Update the set of basic indices (replace the index of the leaving
## variable l_out with that of the entering variable j_in)
function update_B_ind!(B_ind, l_out, j_in)
    # TODO: add your code here
    B_ind[l_out] = j_in
end

## Compute the new inverse basis B^-1 by performing elementary row
## operations on [B^-1 u] (pivot row index is l_out, the vector u is trans-
## formed into a unit vector with u(l_out) = 1 and u(i) = 0 for other i).
function update_invB!(invB, u, l_out)
    # TODO: add your code here
    for i =1:size(invB)[1]
        if i != l_out
           invB[i,:] = invB[i,:] + (-(u[i]/u[l_out])) * invB[l_out,:]
        end
    end
    invB[l_out,:] = invB[l_out,:] / u[l_out]
end;

Using the functions defined above, we can implement another function for running a single iteration of the revised simplex algorithm.

In [27]:
function revised_simplex_iteration(A, c, B_ind, x, cᵣ, invB, rule, ϵ)
    ## Find all variables with negative reduces costs; the column indices j
    ## of these variables are stored in vector J
    J = findall(vec(cᵣ) .< -ϵ)  ## We use vec(c_r) since findall is type sensitive and might return CartesianIndices instead.

    j_in = select_entering_variable(J, cᵣ, rule)

    u = calculate_reverse_basic_directions(A, invB, j_in)

    ## Find all row indices, stored in vector I, for which the elements of u
    ## are positive.
    I = findall(vec(u) .> ϵ)

    ## In case no element of u is positive (i.e., I is empty), the problem
    ## is unbounded and the optimal cost is -infinity
    ## The final return parameter is a flag telling the main function the outcome of 
    ## this iteration: -1 for unboundedness, 1 for successfully finishing the iteration
    if (isempty(I))
        return B_ind, x, cᵣ, invB, -1
    end

    θ = calculate_step_length(x, B_ind, u, I)

    l_out = select_leaving_variable(B_ind, x, u, I, θ)

    update_x!(x, j_in, B_ind, θ, u)

    update_B_ind!(B_ind, l_out, j_in)

    update_invB!(invB, u, l_out)
    
    cᵣ = calculate_cr(A, c, B_ind, invB)
    
    return B_ind, x, cᵣ, invB, 1
end;

This cell contains the function for the revised simplex method. You do not need to change anything here, but it might be good to understand the main idea of the code nevertheless.

In [28]:
## Revised Simplex

# Naive implementation of the primal revised Simplex. Solves a linear
# programming problem of the form
#
#   min c'x
#   s.t. Ax  = b
#         x >= 0
#
# The function input parameters are the following:
#     A: The constraint matrix
#     b: The RHS (right-hand side) vector
#     c: The vector of cost coefficients
#     B_ind: The column indices of the basic variables corresponding to an
#        initial basic feasible solution
#     rule: an optional keyword argument defining the pivoting rule for 
#        choosing the entering variable. Options are :bland and :dantzig, 
#        default is :bland
#
# The function returns:
#     x_opt:  Decision variable values at the optimal solution
#     f_opt:  Objective function value at the optimal solution
#     x_hist: Decision variable values at each iteration
#     c_hist: Reduced costs at each iteration
#     B_hist: Column indices of the basic variables at each iteration
#     iter:   Number of iterations before termination
#
# Usage: x_opt, f_opt, x_hist, c_hist, B_hist, iter = revised_simplex(A,b,c,B)

function revised_simplex(A_orig, b_orig, c, B_ind_start; rule = :bland)
    ## Error control
    
    ## If a constraint has negative RHS, multiply the constraint by -1
    ## This way, b>=0
    A = zeros(size(A_orig))
    b = zeros(size(b_orig))
    for i = 1:length(b_orig)
        if b_orig[i] < 0
            A[i,:] = -A_orig[i,:]
            b[i] = -b_orig[i]
        else
            A[i,:] = A_orig[i,:]
            b[i] = b_orig[i]
        end
    end

    ## Check the keyword argument
    if rule != :dantzig && rule != :bland
         println("error: rule $(rule) not known, defaulting to Bland.")
         rule = :bland 
    end
    
    ## Initialization phase
    
    ## Initialize the vector of decision variables
    x = zeros(length(c))
    
    ## Construct the basis matrix B according to the column indices of the basic
    ## variables in B_ind and compute its inverse (denoted by invB)
    B_ind = B_ind_start
    B = zeros(length(B_ind), length(B_ind))
    
    B = A[:, B_ind]             # Basis matrix, take the columns of A with indices in B_ind
    invB = inv(B)               # Inverse of the basis matrix
    x[B_ind] = invB * b;        # Calculate x_B for initial basis

    k_max = 50        # At most n_max iterations
    k = 0             # Counter for the while loop
    ϵ = 1E-9          # A small numerical tolerance, explained below: 
                      # Comparisons like x>=0 can fail if x=-1E-10,
                      # when x is practically zero, with some numerical inaccuracy
                      # You can try asking your computer if 0.1+0.2 == 0.3 holds 

    x_hist = zeros(k_max+1,length(x))
    c_hist = zeros(k_max+1,length(c))
    B_hist = zeros(k_max+1,length(B_ind))

    ## Iteration phase
    
    cᵣ = calculate_cr(A, c, B_ind, invB)
    update_history!(c_hist, cᵣ, x_hist, x, B_hist, B_ind, 1)
    
    while iteration_condition(cᵣ, ϵ)
        @assert k<k_max "Maximum number of iterations reached"

        B_ind, x, cᵣ, invB, returnflag = revised_simplex_iteration(A, c, B_ind, x, cᵣ, invB, rule, ϵ)
        
        k += 1                 # Increase counter
        
        if returnflag == -1    # Problem is unbounded
            f_opt = "-inf"     # Optimal objective function cost = -inf
            x_opt = []         # Produce empty vector []
            return x_opt, f_opt, x_hist[1:k,:], c_hist[1:k,:], B_hist[1:k,:], k
        else
            update_history!(c_hist, cᵣ, x_hist, x, B_hist, B_ind, k+1)
        end
    end
    
    ## Optimal solution was found if we manage to break out of the while loop
    f_opt = dot(c,x)        # Optimal objective function cost
    x_opt = x               # Values of the decision variables in the optimum
    return x_opt, f_opt, x_hist[1:k+1,:], c_hist[1:k+1,:], B_hist[1:k+1,:], k 
end;
# End function

## Two-phase simplex
Finally, we can use this to implement the two-phase simplex method. Again, you only need to implement the parts with "TODO" -comments. In order to reduce the workload of this assignment, we do not consider the preprocessing for a degenerate auxiliary problem.

In [29]:
## Implements a general two-phase simplex method. 
## Presumes that for feasible problems, all artificial variables are zero.

function two_phase(A, b, c; rule=:bland)
    
    ## Augmenting problem to auxiliary form:
    ## [0 1] [x z]'
    ## [A Z] [x z]' = b
    ## [x z] ≥ 0 
    
    m,n = size(A)
    # TODO: add your code here
    A_ = hcat(A, Matrix(I,m,m))
    c_ = vcat(zeros(size(c)[1]), ones(m))
    B_ind_start = collect((n+1):(n+m)) #collect((n+1):m)
    
    ## Phase 1 problem
    x_opt, f_opt, x_hist, c_hist, B_hist, k1 = revised_simplex(A_, b, c_, B_ind_start, rule=rule)

    if !iszero(f_opt)                                       # Infeasible problem
        println("error: Infeasible problem. z=$(f_opt).")
        f_opt = "+inf"
        x_opt = []             
        return x_opt, f_opt, x_hist, c_hist, B_hist, k1 
    elseif any(j -> j > n, B_hist[k1+1,:])                  # Feasible but with basic artificial variable
        println("warning: The problem is feasible, however the artificial var. x$(j) is basic in the optimal;")
        println("Revise the basis to remove it before calling revised_simplex() with obtained B_ind.")
        return x_opt, f_opt, x_hist, c_hist, B_hist, k1
    else                                                    # Proceed to phase 2 
        B_ind = convert.(Int64, B_hist[k1+1,:])                
        x_opt, f_opt, x_hist, c_hist, B_hist, k2 = revised_simplex(A, b, c, B_ind, rule=rule)
        return x_opt, f_opt, x_hist, c_hist, B_hist, k1+k2
    end
end;

## Testing your code

You can use the cells below to test your functions and see if you obtain the same result from Exercise 4.4 that was solved in the exercise session.

In [30]:
## Test problem (Exercise 4.4)

## Maximize 5x1 + x2
## s.t.     2x1 + x2  >= 5
##                x2  >= 1
##          2x1 + 3x2 <= 12
##           x1,   x2 >= 0

## becomes

## Minimize -5x1 -  x2
## s.t.      2x1 +  x2 - x3           = 5
##                  x2      - x4      = 1
##           2x1 + 3x2           + x5 = 12
##            x1,   x2,  x3,  x4,  x5 >= 0


A = [2  1 -1  0  0;
     0  1  0 -1  0;
     2  3  0  0  1];
b = [5, 1, 12];
c = -[5, 1, 0, 0, 0];

ϵ = 1E-9

# Basis was obtained from solving the auxiliary problem in the exercise, 
# these tests only consider the second phase
B_ind = [1,2,4]
invB = inv(A[:,B_ind])
x = [0.75, 3.5, 0.0, 2.5, 0.0]

println("Initial basis is $(B_ind) with x equal to $(x)\n")

Initial basis is [1, 2, 4] with x equal to [0.75, 3.5, 0.0, 2.5, 0.0]



In [31]:
## Testing calculate_cr
cᵣ = calculate_cr(A, c, B_ind, invB)
println("Reduced costs are $(cᵣ)")
println("Should be         [0.0 0.0 -3.25 0.0 -0.75]")

Reduced costs are [0.0 0.0 -3.25 0.0 -0.75]
Should be         [0.0 0.0 -3.25 0.0 -0.75]


In [32]:
## Testing iteration_condition()
cond = iteration_condition(cᵣ, ϵ)
println("The smallest reduced cost is $(minimum(cᵣ)) and the algorithm ", if cond "continues" else "terminates with an optimal solution" end, ", should continue")

The smallest reduced cost is -3.25 and the algorithm continues, should continue


In [33]:
J = findall(vec(cᵣ) .< -ϵ)       # We use vec(c_r) since findall is type sensitive and might return CartesianIndices instead.

## Testing select_entering_variable()
rule = :bland
j_in = select_entering_variable(J, cᵣ, rule)
println("Entering variable with Bland's rule is x_$(j_in), should be x_3")

rule = :dantzig
j_in = select_entering_variable(J, cᵣ, rule)
println("Entering variable with Dantzig's rule is x_$(j_in), should be x_3")

Entering variable with Bland's rule is x_3, should be x_3
Entering variable with Dantzig's rule is x_3, should be x_3


In [34]:
## Testing calculate_reverse_basic_directions()
u = calculate_reverse_basic_directions(A, invB, j_in)
println("Reverse basic directions (the tableau column corresponding to the entering variable) are $(u), should be [-0.75, 0.5, 0.5]")

Reverse basic directions (the tableau column corresponding to the entering variable) are [-0.75, 0.5, 0.5], should be [-0.75, 0.5, 0.5]


In [35]:
Ir = findall(vec(u) .> ϵ)

## Testing calculate_step_length()
θ = calculate_step_length(x, B_ind, u, Ir)
println("Step length is $(θ), should be 5.0")

Step length is 5.0, should be 5.0


In [36]:
## Testing select_leaving_variable()
l_out = select_leaving_variable(B_ind, x, u, Ir, θ)
println("Leaving variable is x_$(B_ind[l_out]) (row $(l_out) of the tableau), should be x_4 (row 3)\n")

Leaving variable is x_4 (row 3 of the tableau), should be x_4 (row 3)



In [37]:
## Testing update_x!()
update_x!(x, j_in, B_ind, θ, u)
println("New vector x is $(x)")
println("Should be       [4.5, 1.0, 5.0, 0.0, 0.0]")

New vector x is [4.5, 1.0, 5.0, 0.0, 0.0]
Should be       [4.5, 1.0, 5.0, 0.0, 0.0]


In [38]:
## Testing update_B_ind!()
update_B_ind!(B_ind, l_out, j_in)
println("New basis is $(B_ind)")
println("Should be    [1, 2, 3]")

New basis is [1, 2, 3]
Should be    [1, 2, 3]


In [39]:
## Testing update_invB!()
update_invB!(invB, u, l_out)
println("New invB is ", invB)
println("Should be   ", [0.0 -1.5 0.5; 0.0 1.0 0.0; -1.0 -2.0 1.0], "\n")

New invB is [0.0 -1.5 0.5; 0.0 1.0 0.0; -1.0 -2.0 1.0]
Should be   [0.0 -1.5 0.5; 0.0 1.0 0.0; -1.0 -2.0 1.0]



In [40]:
cᵣ = calculate_cr(A, c, B_ind, invB)
println("Reduced costs are $(cᵣ)")
println("Should be         [0.0 0.0 0.0 6.5 2.5]")

Reduced costs are [0.0 0.0 0.0 6.5 2.5]
Should be         [0.0 0.0 0.0 6.5 2.5]


In [41]:
cond = iteration_condition(cᵣ, ϵ)
println("The smallest reduced cost is $(minimum(cᵣ)) and the algorithm ", if cond "continues" else "terminates with an optimal solution" end, ", should terminate")

The smallest reduced cost is 0.0 and the algorithm terminates with an optimal solution, should terminate


In [42]:
x_opt, f_opt, x_hist, c_hist, B_hist, k = revised_simplex(A, b, c, [1,2,4]);
x_opt, f_opt, x_hist, c_hist, B_hist, k1 = two_phase(A, b, c)
x_opt, f_opt, x_hist, c_hist, B_hist, k2 = two_phase(A, b, c, rule=:dantzig)

# It should print (3, 4, [4.5, 1.0, 5.0, 0.0, 0.0], 23.5)
(k1, k2, x_opt, -f_opt)

(3, 4, [4.5, 1.0, 5.0, 0.0, 0.0], 23.5)

In [43]:
# Item 3.a
# Standard form of the problem
# min. -10x1 - 12x2 - 12x3
# s.t.
#      x1 + 2x2 + 2x3 + x4 = 20
#      2x1 + x2 + 2x3 + x5 = 20
#      2x1 + 2x2 + x2 + x6 = 20
#      x1,...,x6 ≥ 0

# TODO: add your code here
A = [1 2 2 1 0 0;
     2 1 2 0 1 0;
     2 2 1 0 0 1];
b = [20, 20, 20];
c = -[10, 12, 12, 0, 0, 0];

x_opt_B, f_opt_B, x_hist_B, c_hist_B, B_hist_B, k_B = two_phase(A, b, c)
println((k_B, x_opt_B, -f_opt_B))

x_opt_D, f_opt_D, x_hist_D, c_hist_D, B_hist_D, k_D = two_phase(A, b, c, rule=:dantzig)
println((k_D, x_opt_D, -f_opt_D))

(3, [3.999999999999999, 4.0, 3.9999999999999982, 0.0, 0.0, 0.0], 135.99999999999997)
(3, [3.999999999999999, 4.0, 3.9999999999999982, 0.0, 0.0, 0.0], 135.99999999999997)


In [44]:
# Item 3.b
# Standard form of the problem
# min. -x1 - x2 - x3 - x4
# s.t.
#      x1 + x2 + x3 + x4 + x5 = 20
#      x1 + 2x2 + 3x4 + +x6 = 24
#      2x1 + x2 + 2x3 - x7 = 16
#      2x1 + 3x2 + x3 + x4 = 20
#      x1,...,x7 ≥ 0

# TODO: add your code here
A = [1 1 1 1 1 0 0;
     1 2 0 3 0 1 0;
     2 1 2 0 0 0 -1;
     2 3 1 1 0 0 0];
b = [20, 24, 16, 20];
c = -[1,1,1,1,0,0,0];

x_opt_B, f_opt_B, x_hist_B, c_hist_B, B_hist_B, k_B = two_phase(A, b, c)
println((k_B, x_opt_B, -f_opt_B))

x_opt_D, f_opt_D, x_hist_D, c_hist_D, B_hist_D, k_D = two_phase(A, b, c, rule=:dantzig)
println((k_D, x_opt_D, -f_opt_D))

(7, [0.0, 0.0, 12.0, 8.0, 8.881784197001252e-16, 0.0, 8.000000000000002], 20.0)
(5, [0.0, 0.0, 12.0, 7.999999999999997, 8.881784197001252e-16, 0.0, 8.0], 19.999999999999996)


In [45]:
# Item 3.c
# Standard form of the problem
# min. x1 - 2x2 + 3x3 + x4
# s.t.
#      x1 + 2x2 + 3x3 + x4 + x5 = 15
#      2x1 + x2 + 5x3 - x6 = 10
#      2x1 + 2x2 + x3 + x4 - x7 = 10

# TODO: add your code here
A = [1 2 3 1 1 0 0;
     2 1 5 0 0 -1 0;
     2 1 2 0 0 0 -1];
b = [15, 10, 10];
c = [1, -2, 3, 1, 0, 0, 0];

x_opt_B, f_opt_B, x_hist_B, c_hist_B, B_hist_B, k_B = two_phase(A, b, c)
println((k_B, x_opt_B, f_opt_B))

x_opt_D, f_opt_D, x_hist_D, c_hist_D, B_hist_D, k_D = two_phase(A, b, c, rule=:dantzig)
println((k_D, x_opt_D, f_opt_D))

(3, [1.666666666666666, 6.666666666666667, 0.0, 0.0, 0.0, 0.0, 0.0], -11.666666666666668)
(4, [1.666666666666666, 6.666666666666667, 0.0, 0.0, 0.0, 0.0, 0.0], -11.666666666666668)
